In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import r2_score
import folium

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()
numCores = multiprocessing.cpu_count()

In [ ]:
lonMin = (float)(config['SEC_ORDER_VISUAL']['LON_MIN'])
lonMax = (float)(config['SEC_ORDER_VISUAL']['LON_MAX'])

latMin = (float)(config['SEC_ORDER_VISUAL']['LAT_MIN'])
latMax = (float)(config['SEC_ORDER_VISUAL']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['SEC_ORDER_VISUAL']['INCR_STEP'])
incrRes = (int)(config['SEC_ORDER_VISUAL']['INCR_RES'])

boundary = (float)(config['SEC_ORDER_VISUAL']['BOUNDARY'])

fileDir = "../Data/M120_50_M119_00_33_90_34_44/Display/"

In [ ]:
#this is useful for mapping
#probabilistic location
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
lonCen = (lonMin + lonMax)/2
latCen = (latMin + latMax)/2
print(lonCen)
print(latCen)


In [ ]:
true1File = fileDir + "Traj1" + "/True.csv"
true1Traj, _ = aISDM.load_data_from_csv(true1File)
print(true1Traj.shape)

true2File = fileDir + "Traj2" + "/True.csv"
true2Traj, _ = aISDM.load_data_from_csv(true2File)
print(true2Traj.shape)

true3File = fileDir + "Traj3" + "/True.csv"
true3Traj, _ = aISDM.load_data_from_csv(true3File)
print(true3Traj.shape)

true4File = fileDir + "Traj4" + "/True.csv"
true4Traj, _ = aISDM.load_data_from_csv(true4File)
print(true4Traj.shape)

true5File = fileDir + "Traj5" + "/True.csv"
true5Traj, _ = aISDM.load_data_from_csv(true5File)
print(true5Traj.shape)

true6File = fileDir + "Traj6" + "/True.csv"
true6Traj, _ = aISDM.load_data_from_csv(true6File)
print(true6Traj.shape)

In [ ]:
def add_small_vessel_to_map(lonVal, latVal, colorValue = '#B03060'):
    vesselColor = colorValue
    folium.CircleMarker(
        location=[latVal, lonVal],
        radius=2,
        popup='Laurelhurst Park',
        color= vesselColor,
        fill=True,
        fill_color=vesselColor
    ).add_to(bGMap)

def add_vessel_to_map(lonVal, latVal, colorValue = '#B03060'):
    vesselColor = colorValue
    folium.CircleMarker(
        location=[latVal, lonVal],
        radius=2,
        popup='Laurelhurst Park',
        color= vesselColor,
        fill=True,
        fill_color=vesselColor
    ).add_to(bGMap)
    
    folium.CircleMarker(
        location=[latVal, lonVal],
        radius=3,
        popup='Laurelhurst Park',
        color=vesselColor,
        fill=True,
        fill_color=vesselColor
    ).add_to(bGMap)
    
    folium.CircleMarker(
        location=[latVal, lonVal],
        radius=5,
        popup='Laurelhurst Park',
        color=vesselColor,
        fill=True,
        fill_color=vesselColor
    ).add_to(bGMap)

In [ ]:
#write a function for the top 5 locations
#first read every trajectory file
#from the stored array
probTrajDir = fileDir + "Traj1/"
probTrajList = []

for i in range(true1Traj.shape[0]):
    probTrajFile = probTrajDir + str(i) + '.npy'
    probTrajList.append(np.load(probTrajFile))
    
probTrajDir1 = fileDir + "Traj2/"
probTrajList1 = []

for i in range(true2Traj.shape[0]):
    probTrajFile = probTrajDir1 + str(i) + '.npy'
    probTrajList1.append(np.load(probTrajFile))
    
probTrajDir2 = fileDir + "Traj3/"
probTrajList2 = []

for i in range(true3Traj.shape[0]):
    probTrajFile = probTrajDir2 + str(i) + '.npy'
    probTrajList2.append(np.load(probTrajFile))
    
probTrajDir3 = fileDir + "Traj4/"
probTrajList3 = []

for i in range(true4Traj.shape[0]):
    probTrajFile = probTrajDir3 + str(i) + '.npy'
    probTrajList3.append(np.load(probTrajFile))
    
probTrajDir4 = fileDir + "Traj5/"
probTrajList4 = []

for i in range(true5Traj.shape[0]):
    probTrajFile = probTrajDir4 + str(i) + '.npy'
    probTrajList4.append(np.load(probTrajFile))
    
probTrajDir5 = fileDir + "Traj6/"
probTrajList5 = []

for i in range(true5Traj.shape[0]):
    probTrajFile = probTrajDir5 + str(i) + '.npy'
    probTrajList5.append(np.load(probTrajFile))

In [ ]:
def get_lon_lat_from_idx(idx):
    tempLonMin = boundaryArray[idx][0]
    tempLonMax = boundaryArray[idx][1]
    tempLatMin = boundaryArray[idx][2]
    tempLatMax = boundaryArray[idx][3]
#     print(tempLonMin,tempLonMax,tempLatMin,tempLatMax)
    return (tempLonMin+tempLonMax)/2,(tempLatMin+tempLatMax)/2

In [ ]:
#compute top 5 or top 10 
#scale them up
#make list of corresponding LONG and LAT
def get_list_lon_lat_for_heat_map(probArray):
    #get top 10 values indexes
    top10Idx = probArray.argsort()[-10:][::-1]
#     print(top10Idx)
    #get those top 10 values
    top10Val = probArray[top10Idx]
#     print(top10Val)
    #get array of top 10 bool value
    #useful for identifying non zero values
    top10NZBool = top10Val > 0
#     print(top10NZBool)

    #get indexes of non zero top 10 values
    #it can be less than 10
    top10IdxNZ = top10Idx[top10NZBool]
#     print(top10IdxNZ)

    #get those non zero values
    top10NZ = probArray[top10IdxNZ]
#     print(top10NZ)
    #normalise it
    normTop10NZ = top10NZ/np.sum(top10NZ)
#     print(normTop10NZ)
    #multiply by 100
    #so the sum will add up to 100
    normTop10NZ = normTop10NZ * 100
#     print(np.sum(normTop10NZ))
    normTop10NZ = np.around(normTop10NZ,2)
    #now for index location
    lonLatList = []
    for idx in range(top10IdxNZ.shape[0]):
#         print(top10IdxNZ[idx])
#         print(normTop10NZ[idx])
        lonCen, latCen = get_lon_lat_from_idx(top10IdxNZ[idx])
#         print(normTop10NZ[idx])
        for count in range(int(normTop10NZ[idx])):
            lonLatList.append([latCen,lonCen])
    return lonLatList

#useful for putting marker on the map
def get_top_one_lon_lat(probArray):
    #get top 10 values indexes
    top = probArray.argsort()[-1:][::-1]
    lonCen, latCen = get_lon_lat_from_idx(top[0])
    return lonCen,latCen
        
heatData = get_list_lon_lat_for_heat_map(probTrajList[0])
lonTop,latTop = get_top_one_lon_lat(probTrajList[0])

In [ ]:
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup, LayerControl, Map, Marker

#the following routine can go 
#under for loop 
#Now construct map
true1TrajLat = true1Traj['LAT']
true1TrajLon = true1Traj['LON']

true2TrajLat = true2Traj['LAT']
true2TrajLon = true2Traj['LON']

true3TrajLat = true3Traj['LAT']
true3TrajLon = true3Traj['LON']

true4TrajLat = true4Traj['LAT']
true4TrajLon = true4Traj['LON']

true5TrajLat = true5Traj['LAT']
true5TrajLon = true5Traj['LON']

true6TrajLat = true6Traj['LAT']
true6TrajLon = true6Traj['LON']

#get rid of starting 12 trajectories
startFrame = 12
maxFrame = 90
for frameNum in range(startFrame,maxFrame):
    bGMap = folium.Map(location=[latCen, lonCen+0.5], zoom_start = 8 \
                       , min_zoom=8 \
                       , max_zoom=8 \
                       , tiles='Stamen Watercolor' \
                       , min_lat = (latMin-boundary) \
                       , max_lat = (latMax+boundary) \
                       , min_lon = (lonMin-boundary) \
                       , max_lon = (lonMax+boundary) \
                      )


    heatData = [[true1TrajLat.iloc[frameNum],true1TrajLon.iloc[frameNum]]]

    if(frameNum < true1TrajLon.shape[0]):
        add_vessel_to_map(true1TrajLon.iloc[frameNum],true1TrajLat.iloc[frameNum], colorValue = '#FF2C34')
    if(frameNum < true2TrajLon.shape[0]):
        add_vessel_to_map(true2TrajLon.iloc[frameNum],true2TrajLat.iloc[frameNum], colorValue = '#81F791')
    if(frameNum < true3TrajLon.shape[0]):
        add_vessel_to_map(true3TrajLon.iloc[frameNum],true3TrajLat.iloc[frameNum], colorValue = '#2630B3')
    if(frameNum < true4TrajLon.shape[0]):
        add_vessel_to_map(true4TrajLon.iloc[frameNum],true4TrajLat.iloc[frameNum], colorValue = '#FFFF3E')    
    if(frameNum < true5TrajLon.shape[0]):
        add_vessel_to_map(true5TrajLon.iloc[frameNum],true5TrajLat.iloc[frameNum], colorValue = '#F04EF0')
    if(frameNum < true6TrajLon.shape[0]):
        add_vessel_to_map(true6TrajLon.iloc[frameNum],true6TrajLat.iloc[frameNum], colorValue = '#F7A1A1')
    

    #save the map into output directory
    htmlName = "../Data/M120_50_M119_00_33_90_34_44/Display/HTML/"+str(frameNum-startFrame)+".html"
    bGMap
    bGMap.save(outfile=htmlName)

In [ ]:
#Function assumes basemap instance is present
#probState is numpy array 
#whose top 10 values would be used for blurring effect
def add_probable_vessel_to_map(probState, colval = '#FF2C34'):
    heatData = get_list_lon_lat_for_heat_map(probState)
    tempLonTop,tempLatTop = get_top_one_lon_lat(probState)

    HeatMap(heatData \
        , radius=7 \
        , blur=6 \
        ,gradient={0.0: colval, 1: colval} \
       ).add_to(bGMap)
#     add_small_vessel_to_map(tempLonTop,tempLatTop)

In [ ]:
startFrame = 12
maxFrame = 90
for frameNum in range(startFrame,maxFrame):
    bGMap = folium.Map(location=[latCen, lonCen+0.5], zoom_start = 8 \
                       , min_zoom=8 \
                       , max_zoom=8 \
                       , tiles='Stamen Watercolor' \
                       , min_lat = (latMin-boundary) \
                       , max_lat = (latMax+boundary) \
                       , min_lon = (lonMin-boundary) \
                       , max_lon = (lonMax+boundary) \
                      )
    
    
    if(frameNum < len(probTrajList)):
        add_probable_vessel_to_map(probTrajList[frameNum],'#FF2C34')
        
    if(frameNum < len(probTrajList1)):
        add_probable_vessel_to_map(probTrajList1[frameNum],'#81F791')
    
    if(frameNum < len(probTrajList2)):
        add_probable_vessel_to_map(probTrajList2[frameNum],'#2630B3')
        
    if(frameNum < len(probTrajList3)):
        add_probable_vessel_to_map(probTrajList3[frameNum],'#FFFF3E')
        
    if(frameNum < len(probTrajList4)):
        add_probable_vessel_to_map(probTrajList4[frameNum],'#F04EF0')
        
    if(frameNum < len(probTrajList5)):
        add_probable_vessel_to_map(probTrajList5[frameNum],'#F7A1A1')
    
    #save the map into output directory
    htmlName = "../Data/M120_50_M119_00_33_90_34_44/Display/HTML/"+str(frameNum-startFrame)+"_Prob.html"
    bGMap
    bGMap.save(outfile=htmlName)

In [ ]:
startFrame = 12
maxFrame = 90
for frameNum in range(startFrame,maxFrame):
    bGMap = folium.Map(location=[latCen, lonCen+0.5], zoom_start = 8 \
                       , min_zoom=8 \
                       , max_zoom=8 \
                       , tiles='Stamen Watercolor' \
                       , min_lat = (latMin-boundary) \
                       , max_lat = (latMax+boundary) \
                       , min_lon = (lonMin-boundary) \
                       , max_lon = (lonMax+boundary) \
                      )
    
    if(frameNum < true1TrajLon.shape[0]):
        add_small_vessel_to_map(true1TrajLon.iloc[frameNum],true1TrajLat.iloc[frameNum], colorValue = '#FF2C34')
    if(frameNum < true2TrajLon.shape[0]):
        add_small_vessel_to_map(true2TrajLon.iloc[frameNum],true2TrajLat.iloc[frameNum], colorValue = '#81F791')
    if(frameNum < true3TrajLon.shape[0]):
        add_small_vessel_to_map(true3TrajLon.iloc[frameNum],true3TrajLat.iloc[frameNum], colorValue = '#2630B3')
    if(frameNum < true4TrajLon.shape[0]):
        add_small_vessel_to_map(true4TrajLon.iloc[frameNum],true4TrajLat.iloc[frameNum], colorValue = '#FFFF3E')    
    if(frameNum < true5TrajLon.shape[0]):
        add_small_vessel_to_map(true5TrajLon.iloc[frameNum],true5TrajLat.iloc[frameNum], colorValue = '#F04EF0')
    if(frameNum < true6TrajLon.shape[0]):
        add_small_vessel_to_map(true6TrajLon.iloc[frameNum],true6TrajLat.iloc[frameNum], colorValue = '#F7A1A1')

    
    
    if(frameNum < len(probTrajList)):
        add_probable_vessel_to_map(probTrajList[frameNum],'#FF2C34')
        
    if(frameNum < len(probTrajList1)):
        add_probable_vessel_to_map(probTrajList1[frameNum],'#81F791')
    
    if(frameNum < len(probTrajList2)):
        add_probable_vessel_to_map(probTrajList2[frameNum],'#2630B3')
        
    if(frameNum < len(probTrajList3)):
        add_probable_vessel_to_map(probTrajList3[frameNum],'#FFFF3E')
        
    if(frameNum < len(probTrajList4)):
        add_probable_vessel_to_map(probTrajList4[frameNum],'#F04EF0')
        
    if(frameNum < len(probTrajList5)):
        add_probable_vessel_to_map(probTrajList5[frameNum],'#F7A1A1')
    
    #save the map into output directory
    htmlName = "../Data/M120_50_M119_00_33_90_34_44/Display/HTML/"+str(frameNum-startFrame)+"_True_Prob.html"
    bGMap
    bGMap.save(outfile=htmlName)